### Introduction

UMAP is a method that can allow the spatial recognition of a relationship between the input data and the associated labels. It reduced the dimensions of the data into 2D and classifies them into

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# For feature selection
from scipy import stats
from statsmodels.stats.multitest import multipletests
from sklearn.feature_selection import SelectKBest, f_classif

# For dimensionality reduction
import umap

# For optional scaling
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore') # Suppress common warnings for cleaner output